## Assignment 1
### Name: Saikat Moi
### Roll Number: 20CS10050

In [214]:
# import all the necessary libraries here
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix

In [215]:
data = pd.read_csv('../../dataset/cross-validation.csv')
print(data.shape) 

(614, 13)


In [216]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Load your dataset
data = pd.read_csv('../../dataset/cross-validation.csv')

# Remove rows with any missing values
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
data['Dependents'] = data['Dependents'].replace('3+', 4)

# Map 'Y' to 1 and 'N' to 0 in the 'Loan_Status' column
data['Loan_Status'] = data['Loan_Status'].map({'Y': 1, 'N': 0})

# Separate features (X) and labels (y)
X = data.drop(['Loan_Status', 'Loan_ID'], axis=1)  # Features
y = data['Loan_Status']  # Labels


In [217]:
print(data)

      Loan_ID  Gender Married Dependents     Education Self_Employed  \
0    LP001003    Male     Yes          1      Graduate            No   
1    LP001005    Male     Yes          0      Graduate           Yes   
2    LP001006    Male     Yes          0  Not Graduate            No   
3    LP001008    Male      No          0      Graduate            No   
4    LP001011    Male     Yes          2      Graduate           Yes   
..        ...     ...     ...        ...           ...           ...   
475  LP002978  Female      No          0      Graduate            No   
476  LP002979    Male     Yes          4      Graduate            No   
477  LP002983    Male     Yes          1      Graduate            No   
478  LP002984    Male     Yes          2      Graduate            No   
479  LP002990  Female      No          0      Graduate           Yes   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0               4583             1508.0       128.0           

In [218]:
X_encoded = pd.get_dummies(X, drop_first=True)

# Split the dataset into 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Normalize/Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [219]:
logistic_model = LogisticRegression(solver='saga', penalty=None, random_state=42)
logistic_model.fit(X_train, y_train)


LogisticRegression(penalty=None, random_state=42, solver='saga')

In [220]:
import numpy as np

k = 5

accuracy_scores = []
precision_scores = []
recall_scores = []

# Calculate the size of each fold
fold_size = len(X_train) // k

# Perform k-fold cross-validation
for i in range(k):
    # Determine the start and end indices for the current fold
    start = i * fold_size
    end = (i + 1) * fold_size
    
    # Split the data into training and validation sets for the current fold
    X_val_fold = X_train[start:end]
    y_val_fold = y_train[start:end]
    
    # Create training data by excluding the current fold
    X_train_fold = np.concatenate([X_train[:start], X_train[end:]], axis=0)
    y_train_fold = np.concatenate([y_train[:start], y_train[end:]], axis=0)
    
    # Train the model on the training data
    logistic_model.fit(X_train_fold, y_train_fold)
    
    # Make predictions on the validation data
    y_pred_fold = logistic_model.predict(X_val_fold)
    
    # Compute accuracy for this fold
    accuracy_fold = np.mean(y_pred_fold == y_val_fold)
    accuracy_scores.append(accuracy_fold)
    
    # Compute precision and recall for class 1 (good) for this fold
    from sklearn.metrics import precision_score, recall_score
    precision_fold = precision_score(y_val_fold, y_pred_fold, pos_label=1)
    recall_fold = recall_score(y_val_fold, y_pred_fold, pos_label=1)
    precision_scores.append(precision_fold)
    recall_scores.append(recall_fold)


mean_accuracy = np.mean(accuracy_scores)
mean_precision = np.mean(precision_scores)
mean_recall = np.mean(recall_scores)


print("Mean Accuracy:", mean_accuracy)
print("Mean Precision:", mean_precision)
print("Mean Recall:", mean_recall)


Mean Accuracy: 0.8026315789473685
Mean Precision: 0.7918648230022394
Mean Recall: 0.9697142103179839
